In [1]:
from scipy.sparse import load_npz

# загружаем заранее подготовленные данные
user_item_matrix = load_npz("/data/other/user_item_lastfm.npz")

In [2]:
user_item_matrix

<359337x160163 sparse matrix of type '<class 'numpy.float16'>'
	with 17332977 stored elements in COOrdinate format>

In [3]:
import numpy as np

# делим разреженную матрицу на обучающую и тестовую
total_len = user_item_matrix.data.size
train_len = int(total_len * 0.8)
all_indices = np.arange(total_len)
np.random.seed(42)
train_indices = np.random.choice(all_indices, train_len, replace=False)
train_mask = np.in1d(all_indices, train_indices)

In [4]:
print(all_indices)
print(train_indices)
print(train_mask)
print(len(train_mask))

[       0        1        2 ... 17332974 17332975 17332976]
[ 4343353 10204053  1135877 ...  3975992  7428974  3148521]
[ True False  True ... False  True  True]
17332977


In [5]:
from scipy.sparse import coo_matrix

def get_masked(arr, mask):
    return coo_matrix(
        (
            [np.float32(item) for item in arr.data[mask]],
            (arr.row[mask], arr.col[mask])
        ),
        arr.shape
    )

In [6]:
train_csr = get_masked(user_item_matrix, train_mask).tocsr()
# для обучения нужна item*user матрица
train = train_csr.T
test_coo = get_masked(user_item_matrix, ~train_mask)
test_csr = test_coo.tocsr()

In [7]:
from implicit.als import AlternatingLeastSquares
import os

In [9]:
%%time

# обучимся на дефолтных параметрах
als = AlternatingLeastSquares()
als.fit(train)

CPU times: user 6min 47s, sys: 330 ms, total: 6min 48s
Wall time: 6min 49s


In [10]:
import pickle

pickle_filename = "/data/other/implicit_top50.pkl"
users = set(test_coo.row)

In [11]:
%%time

def get_recs(users, model):
    return {
        user: model.recommend(userid=user, user_items=train_csr, N=50)
        for user in users
    }

# посчитаем по 50 рекомендаций для каждого пользователя из тестовой выборки
recs = get_recs(users, als)
# сохраним предрасчёт рекомендаций
with open(pickle_filename, "wb") as f:
    pickle.dump(recs, f)

CPU times: user 1h 58min 53s, sys: 32.5 s, total: 1h 59min 26s
Wall time: 1h 59min 57s


In [12]:
# загрузим сохранённый предрасчёт
with open(pickle_filename, "rb") as f:
    recs = pickle.load(f)

In [15]:
recs

{0: [(118151, 0.8379309),
  (116641, 0.77153575),
  (100396, 0.75668216),
  (17225, 0.7343616),
  (83786, 0.70603746),
  (77910, 0.65576),
  (132928, 0.6183412),
  (107658, 0.6059146),
  (40, 0.5896498),
  (22967, 0.56594723),
  (46299, 0.52153224),
  (124387, 0.49963117),
  (145758, 0.49715385),
  (31, 0.4859691),
  (73312, 0.43803713),
  (16580, 0.43783188),
  (56281, 0.43485647),
  (142900, 0.41661838),
  (113261, 0.39327177),
  (5752, 0.38396037),
  (102109, 0.37817222),
  (30687, 0.37579495),
  (33267, 0.36931857),
  (151478, 0.36532286),
  (4744, 0.36211187),
  (89921, 0.3570806),
  (34493, 0.35676906),
  (103464, 0.3545475),
  (41466, 0.35319635),
  (150223, 0.34960657),
  (5691, 0.33981702),
  (8824, 0.33556673),
  (85892, 0.32055137),
  (77645, 0.3142761),
  (36141, 0.30829674),
  (157686, 0.30436844),
  (73788, 0.30089036),
  (146624, 0.29455537),
  (140399, 0.28515312),
  (131608, 0.28240862),
  (70103, 0.27724132),
  (76314, 0.27513596),
  (64747, 0.27342758),
  (63327, 0.2

In [21]:
test_csr[1].indices

array([  2935,  16778,  36436,  39805,  55629,  59246,  98344, 116170],
      dtype=int32)

In [14]:
%%time

def hitrate(k, recs, users):
    hits = 0
    for user in users:
        if recs[user]:
            rec_items, _ = zip(*recs[user])
            hits += len(set(rec_items[:k]).intersection(set(test_csr[user].indices))) > 0
    return hits / len(users)

print("hitrate@50:", hitrate(50, recs, users))
print("hitrate@20:", hitrate(20, recs, users))
print("hitrate@10:", hitrate(10, recs, users))
print("hitrate@5:", hitrate(5, recs, users))
print("hitrate@1:", hitrate(1, recs, users))

# сравним с бейзлайном
# hitrate@100 = 0.9254293323537515
# hitrate@50 = 0.869095027787286
# hitrate@20 = 0.7547928546183666
# hitrate@10 = 0.6508709094805155
# hitrate@5 = 0.5333739637165112
# hitrate@1 = 0.21251638434116163

hitrate@50: 0.9152947010089867
hitrate@20: 0.8186943901229615
hitrate@10: 0.7076612790898505
hitrate@5: 0.5739078596221453
hitrate@1: 0.2714486285914857
CPU times: user 5min 9s, sys: 90 ms, total: 5min 10s
Wall time: 5min 16s


Функция для расчёта **precision**:

**$precision = \frac {TP}{TP+FP}$**

Для рассчета напишем функцию, в которой будем считать следующим образом:
**TP** - количество угаданных нами композиций (смотрим пересечение предсказанного нами топа с профилем пользователя)
**TP+FP** - берем меньшее из чисел количество композиций, которые слушал пользователь, или число предсказанных композиций

In [33]:
%%time

def precision(k, recs, users):
    #Переменная для подсчета попаданий
    hits = 0
    #Переменная для подсчета TF+TP
    kol=0
    for user in users:
        if recs[user]:
            rec_items, _ = zip(*recs[user])
            hits += len(set(rec_items[:k]).intersection(set(test_csr[user].indices))) > 0
            l=len(set(test_csr[user].indices))
            if l>=k:
                kol =kol+k
            else:
                kol =kol+l
    return hits / kol

print("precision@50:", precision(50, recs, users))
print("precision@20:", precision(20, recs, users))
print("precision@10:", precision(10, recs, users))
print("precision@5:", precision(5, recs, users))
print("precision@1:", precision(1, recs, users))

precision@50: 0.09478144854791834
precision@20: 0.08482981215575502
precision@10: 0.08284240174873282
precision@5: 0.11657975820301779
precision@1: 0.2714486285914857
CPU times: user 12min 6s, sys: 80 ms, total: 12min 6s
Wall time: 12min 10s


In [8]:
# автор пакета утверждает, что так быстрее
os.environ["OPENBLAS_NUM_THREADS"] = "1"
# обучаемся на тех же параметрах, что и в Spark
als2 = AlternatingLeastSquares(
    factors=10,
    iterations=10,
    regularization=0.1
)

In [9]:
%%time

# обучает быстрее, чем Spark
als2.fit(train)

CPU times: user 47.8 s, sys: 110 ms, total: 47.9 s
Wall time: 48 s


In [10]:
import pickle

pickle_filename2 = "/data/other/implicit_top50_2.pkl"
users = set(test_coo.row)

In [11]:
%%time

def get_recs(users, model):
    return {
        user: model.recommend(userid=user, user_items=train_csr, N=50)
        for user in users
    }

# посчитаем по 50 рекомендаций для каждого пользователя из тестовой выборки
recs = get_recs(users, als2)
# сохраним предрасчёт рекомендаций
with open(pickle_filename2, "wb") as f:
    pickle.dump(recs, f)

CPU times: user 18min 12s, sys: 23.6 s, total: 18min 36s
Wall time: 18min 43s


In [12]:
# загрузим сохранённый предрасчёт
with open(pickle_filename2, "rb") as f:
    recs = pickle.load(f)

In [13]:
%%time

def precision(k, recs, users):
    hits = 0
    kol=0
    for user in users:
        if recs[user]:
            rec_items, _ = zip(*recs[user])
            hits += len(set(rec_items[:k]).intersection(set(test_csr[user].indices))) > 0
            l=len(set(test_csr[user].indices))
            if l>=k:
                kol =kol+k
            else:
                kol =kol+l
    return hits / kol

print("precision@50:", precision(50, recs, users))
print("precision@20:", precision(20, recs, users))
print("precision@10:", precision(10, recs, users))
print("precision@5:", precision(5, recs, users))
print("precision@1:", precision(1, recs, users))

precision@50: 0.08476320287315289
precision@20: 0.07116161624908428
precision@10: 0.0658197804692007
precision@5: 0.08807170726324635
precision@1: 0.18493261350404208
CPU times: user 6min 55s, sys: 260 ms, total: 6min 55s
Wall time: 6min 57s


In [16]:
# автор пакета утверждает, что так быстрее
os.environ["OPENBLAS_NUM_THREADS"] = "1"
# обучаемся на тех же параметрах, что и в Spark
als3 = AlternatingLeastSquares(
    factors=20,
    iterations=20,
    regularization=0.1
)

In [17]:
%%time

# обучает быстрее, чем Spark
als3.fit(train)

CPU times: user 1min 59s, sys: 350 ms, total: 1min 59s
Wall time: 1min 59s


In [18]:
users = set(test_coo.row)

In [19]:
%%time

# посчитаем по 50 рекомендаций для каждого пользователя из тестовой выборки
recs3 = get_recs(users, als3)

CPU times: user 23min 38s, sys: 20 ms, total: 23min 38s
Wall time: 23min 41s


In [20]:
print("precision@50:", precision(50, recs3, users))
print("precision@20:", precision(20, recs3, users))
print("precision@10:", precision(10, recs3, users))
print("precision@5:", precision(5, recs3, users))
print("precision@1:", precision(1, recs3, users))

precision@50: 0.08941765287062879
precision@20: 0.07749217339699682
precision@10: 0.07357663538509852
precision@5: 0.10075911690060746
precision@1: 0.22147843576415005
